# 3. Análise de Regressão dos Dados PISA

Este notebook desenvolve e avalia modelos de regressão para prever o desempenho no PISA.

## Objetivos
1. Preparar features para modelagem
2. Desenvolver modelos de regressão
3. Avaliar performance dos modelos
4. Analisar importância das features
5. Validar pressupostos da regressão

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.metrics import mean_squared_error, r2_score
import statsmodels.api as sm

%matplotlib inline
plt.style.use('seaborn')
pd.set_option('display.max_columns', None)

## 1. Preparação dos Dados

In [ ]:
# Carrega dados
df = pd.read_csv('../data/economics_and_education_dataset_CSV.csv')

# Define features para o modelo
features = [
    'expenditure_on _education_pct_gdp',
    'gdp_per_capita_ppp',
    'gini_index',
    'unemployment',
    'urban_population_pct_total'
]

# Prepara X e y
X = df[features].copy()
y = df['rating']

# Remove linhas com valores ausentes
mask = ~(X.isna().any(axis=1) | y.isna())
X = X[mask]
y = y[mask]

# Normaliza features
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)
X_scaled = pd.DataFrame(X_scaled, columns=X.columns)

print('Shape dos dados após preparação:', X_scaled.shape)

## 2. Regressão Linear Simples

In [ ]:
# Split treino-teste
X_train, X_test, y_train, y_test = train_test_split(
    X_scaled, y, test_size=0.2, random_state=42
)

# Treina modelo
model = LinearRegression()
model.fit(X_train, y_train)

# Faz previsões
y_pred = model.predict(X_test)

# Avalia modelo
r2 = r2_score(y_test, y_pred)
mse = mean_squared_error(y_test, y_pred)

print('Performance do Modelo:')
print(f'R² Score: {r2:.4f}')
print(f'MSE: {mse:.4f}')
print(f'RMSE: {np.sqrt(mse):.4f}')

# Plota resultados
plt.figure(figsize=(10, 6))
plt.scatter(y_test, y_pred, alpha=0.5)
plt.plot([y_test.min(), y_test.max()], [y_test.min(), y_test.max()], 'r--', lw=2)
plt.xlabel('Valores Reais')
plt.ylabel('Valores Previstos')
plt.title('Previsões vs. Valores Reais')
plt.show()

## 3. Análise dos Coeficientes

In [ ]:
# Cria DataFrame com coeficientes
coef_df = pd.DataFrame({
    'Feature': features,
    'Coefficient': model.coef_
}).sort_values('Coefficient', ascending=True)

# Plota importância das features
plt.figure(figsize=(10, 6))
sns.barplot(data=coef_df, x='Coefficient', y='Feature')
plt.title('Importância das Features')
plt.axvline(x=0, color='black', linestyle='--')
plt.show()

print('\nCoeficientes do Modelo:')
print(coef_df)

## 4. Validação Cruzada

In [ ]:
# Realiza validação cruzada
cv_scores = cross_val_score(model, X_scaled, y, cv=5, scoring='r2')

print('Resultados da Validação Cruzada:')
print(f'R² médio: {cv_scores.mean():.4f} (+/- {cv_scores.std() * 2:.4f})')

# Plota distribuição dos scores
plt.figure(figsize=(8, 4))
plt.boxplot(cv_scores)
plt.title('Distribuição dos R² Scores na Validação Cruzada')
plt.show()

## 5. Análise de Resíduos

In [ ]:
# Calcula resíduos
residuals = y_test - y_pred

# Plots de diagnóstico
fig, axes = plt.subplots(2, 2, figsize=(15, 10))

# 1. Distribuição dos resíduos
sns.histplot(residuals, kde=True, ax=axes[0,0])
axes[0,0].set_title('Distribuição dos Resíduos')

# 2. Q-Q Plot
stats.probplot(residuals, dist="norm", plot=axes[0,1])
axes[0,1].set_title('Q-Q Plot dos Resíduos')

# 3. Resíduos vs. Valores Previstos
axes[1,0].scatter(y_pred, residuals, alpha=0.5)
axes[1,0].axhline(y=0, color='r', linestyle='--')
axes[1,0].set_xlabel('Valores Previstos')
axes[1,0].set_ylabel('Resíduos')
axes[1,0].set_title('Resíduos vs. Valores Previstos')

# 4. Resíduos vs. Ordem
axes[1,1].plot(residuals, 'o-', alpha=0.5)
axes[1,1].axhline(y=0, color='r', linestyle='--')
axes[1,1].set_title('Resíduos vs. Ordem')

plt.tight_layout()
plt.show()

# Testes estatísticos
print('Teste de Normalidade dos Resíduos (Shapiro-Wilk):')
stat, p_val = stats.shapiro(residuals)
print(f'Estatística: {stat:.4f}')
print(f'P-valor: {p_val:.4f}')

print('\nTeste de Homocedasticidade (Breusch-Pagan):')
from statsmodels.stats.diagnostic import het_breuschpagan
_, p_val, _, _ = het_breuschpagan(residuals, X_test)
print(f'P-valor: {p_val:.4f}')

## 6. Comparação de Modelos

In [ ]:
# Define modelos para comparação
models = {
    'Linear Regression': LinearRegression(),
    'Ridge': Ridge(alpha=1.0),
    'Lasso': Lasso(alpha=1.0)
}

# Avalia cada modelo
results = []
for name, model in models.items():
    # Validação cruzada
    cv_scores = cross_val_score(model, X_scaled, y, cv=5, scoring='r2')
    
    # Treina modelo completo
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    test_r2 = r2_score(y_test, y_pred)
    test_rmse = np.sqrt(mean_squared_error(y_test, y_pred))
    
    results.append({
        'Model': name,
        'CV R² (mean)': cv_scores.mean(),
        'CV R² (std)': cv_scores.std(),
        'Test R²': test_r2,
        'Test RMSE': test_rmse
    })

# Exibe resultados
results_df = pd.DataFrame(results)
print('Comparação de Modelos:')
print(results_df.round(4))

# Plota comparação
plt.figure(figsize=(10, 6))
sns.barplot(data=results_df, x='Model', y='Test R²')
plt.title('Comparação do R² entre Modelos')
plt.xticks(rotation=45)
plt.show()